In [27]:
import requests
import folium
import pandas as pd
from geopy.geocoders import Nominatim
from flask import Flask

base_map = folium.Map(location=[41.26, -72.95], zoom_start=13.3)

url = 'https://docs.google.com/spreadsheets/d/1EyIJRnLrWkeAJuxLVBxO2eSBMfKiQ2aHYLS3PLvQLLs/export?format=csv'
r = requests.get(url, allow_redirects=True)

open('responses.csv', 'wb').write(r.content)

response_df = pd.read_csv("responses.csv")

geolocator = Nominatim(user_agent="Luke's mapping app", timeout=2)

lat_list = []
long_list = []
add_list = []

for address in response_df['Location']:
    print(address)
    geo = geolocator.geocode(address)
    print(geo)
    lat = geo.latitude
    long = geo.longitude
    lat_list.append(lat)
    long_list.append(long)
    add_list.append(geo)
    
#break up large lines
    
for (columnName, columnData) in response_df.iteritems():
    for row in range(len(columnData)):
        cell = str(columnData.iloc[row])
        count = 0
        overall_count = 0
        for char in cell:
            if count >= 50 and char == ' ':
                newstring = cell[:overall_count] + '<br>' + cell[overall_count:]
                response_df.at[row, columnName] = newstring
                cell = newstring
                count = 0
            count +=1
            overall_count += 1
    
response_df['Latitude'] = lat_list
response_df['Longitude'] = long_list
response_df['Geolocated address'] = add_list

for index, row in response_df.iterrows():
    
    caption_info = f"<b>{row['Name']}</b> <br> <b>Category:</b> {row['Category - What is it?']} <br><b>Type:</b> {row['Type']} <br><b>Location:</b> {row['Location']} <br><b>Notes:</b> {row['Notes']}" 
    
    folium.CircleMarker(
                # Latitude, longitude for each marker
                location=[row['Latitude'], row['Longitude']],
        
                # Size, fill, color of the circle marker
                radius = 10, fill=True, color='orange', #(row['FY18.Max.Population.Count']+1)/25, fill = True, color='orange',
        
                # Text that goes into the popup or tooltip
                tooltip = caption_info 
                  
                 ).add_to(base_map)

html = base_map._repr_html_()

file = open("index.html","w")
file.write(html)
file.close()

18 4th ave, west haven ct
18, 4th Avenue, West Haven, New Haven County, Connecticut, 06516, United States
379 union ave, west haven ct
379, Union Avenue, West Haven, New Haven County, Connecticut, 06516, United States
500 orange ave, west haven ct
500, Orange Avenue, Allingtown, West Haven, New Haven County, Connecticut, 06516, United States
Visual Arts, Film, TV, Media, Technology (gaming, web<br> design...), Graphic Design/Printmaking
           Timestamp   Name  \
0  7/5/2021 17:24:23  mom's   
1  7/5/2021 17:24:51  dad's   
2  7/5/2021 20:00:41  ESUMS   

                              Category - What is it?                   Type  \
0         Literary Arts, Architecture, Culinary Arts      Performance Venue   
1  Visual Arts, Film, TV, Media, Technology (gami...  Landmark/Public Space   
2                 Technology (gaming, web design...)       School/Education   

                        Location      Notes  Explain "Other," if selected  \
0      18 4th ave, west haven ct     asd

In [28]:
base_map